### Tool Depot

In [445]:
#import neccesary libraries
import pandas as pd
import numpy as np
import math

In [446]:
#We will read the libraries and make the accessible
carrier_rates = pd.read_csv('C:/Users/valde/Desktop/Tool Depot/tool-depot/Datasets/Carrier_rates.csv')
historical_orders = pd.read_csv('C:/Users/valde/Desktop/Tool Depot/tool-depot/Datasets/Historical_orders.csv')
historical_shipments = pd.read_csv('C:/Users/valde/Desktop/Tool Depot/tool-depot/Datasets/Historical_shipments.csv')
product_info = pd.read_csv('C:/Users/valde/Desktop/Tool Depot/tool-depot/Datasets/Product_info.csv')

In [447]:
#check of libraries were read properly
carrier_rates_summary = carrier_rates.head()
historical_orders_summary = historical_orders.head()
historical_shipments_summary = historical_shipments.head()
product_info_summary = product_info.head()
display(carrier_rates_summary.head())
display(historical_orders_summary.head())
display(historical_shipments_summary.head())
display(product_info_summary.head())

,Carrier,Supplier,Str Nbr,Cost,Unnamed: 4
0,X,A,1,"$2,540",NaN
1,X,A,2,"$1,640",NaN
2,X,B,1,"$1,200",NaN
3,X,B,2,"$1,200",NaN
4,Y,A,1,$0.12,NaN


,Order Dt,Str Nbr,Supplier,Product Id,Order Qty,Product Weight,Order Weight
0,8/4/2014,1,A,1,"4,338",2.2,"9,544"
1,8/4/2014,1,A,2,"5,006",1.2,"6,007"
2,8/4/2014,2,A,1,"3,600",2.2,"7,920"
3,8/4/2014,2,A,2,"5,173",1.2,"6,208"
4,8/4/2014,1,B,3,"1,890",8.3,"15,687"


,Shipment Date,Str Nbr,Supplier,Weight,Unnamed: 4,Carrier X Cost,Carrier Y Cost
0,2014-08-04,1,A,15551,NaN,2540,1866.12
1,2014-08-04,1,B,15687,NaN,1200,1098.09
2,2014-08-04,2,A,14128,NaN,1640,988.96
3,2014-08-04,2,B,21049,NaN,1200,1262.94
4,2014-08-11,1,A,16671,NaN,2540,2000.52


,Product Id,Product Desc,Package Qty,Weight,Weight Unit of Measure
0,1,Wrench,1,2.2,LB
1,2,Saw,1,1.2,LB
2,3,Drill,1,8.3,LB
3,4,Hammer,1,2.0,LB


Everything seems good, we move on to check if data is clean

The following code will check if data is clean for most datasets

In [448]:
#Check for data type before this to make sure the data type is correct
carrier_rates.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8 entries, 0 to 7
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Carrier     8 non-null      object 
 1   Supplier    8 non-null      object 
 2   Str Nbr     8 non-null      int64  
 3   Cost        8 non-null      object 
 4   Unnamed: 4  0 non-null      float64
dtypes: float64(1), int64(1), object(3)
memory usage: 452.0+ bytes


In [449]:
#Remove empty column from carriers_rates as its empty
carrier_rates = carrier_rates.drop(columns=['Unnamed: 4'])

In [450]:
#Check again if column is not there anymore
carrier_rates.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8 entries, 0 to 7
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Carrier   8 non-null      object
 1   Supplier  8 non-null      object
 2   Str Nbr   8 non-null      int64 
 3   Cost      8 non-null      object
dtypes: int64(1), object(3)
memory usage: 388.0+ bytes


Empty column was removed from dataset

In [451]:
#Check for data type before this to make sure the data type is correct
historical_orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 312 entries, 0 to 311
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Order Dt          312 non-null    object 
 1   Str Nbr           312 non-null    int64  
 2   Supplier          312 non-null    object 
 3   Product Id        312 non-null    int64  
 4    Order Qty        312 non-null    object 
 5    Product Weight   312 non-null    float64
 6    Order Weight     312 non-null    object 
dtypes: float64(1), int64(2), object(4)
memory usage: 17.2+ KB


In [452]:
#Check for data type before this to make sure the data type is correct
historical_shipments.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 208 entries, 0 to 207
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Shipment Date   208 non-null    object 
 1   Str Nbr         208 non-null    int64  
 2   Supplier        208 non-null    object 
 3   Weight          208 non-null    int64  
 4   Unnamed: 4      0 non-null      float64
 5   Carrier X Cost  208 non-null    int64  
 6   Carrier Y Cost  208 non-null    float64
dtypes: float64(2), int64(3), object(2)
memory usage: 11.5+ KB


Empty column has been removed

In [453]:
#Check for data type before this to make sure the data type is correct
product_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 5 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Product Id              4 non-null      int64  
 1   Product Desc            4 non-null      object 
 2   Package Qty             4 non-null      int64  
 3    Weight                 4 non-null      float64
 4   Weight Unit of Measure  4 non-null      object 
dtypes: float64(1), int64(2), object(2)
memory usage: 292.0+ bytes


Since we expect the same number of hammers to be sold compare to the wrench we will take this as a reference for the future sales.

### Step 1

Total number of hammers order per month

In [454]:
#Convert the Order date to datetime
historical_orders['Order Dt'] = pd.to_datetime(historical_orders['Order Dt'])

In [455]:
#Filter data the data from "Wrench" product which is 1
wrench_orders = historical_orders[historical_orders['Product Id'] == 1]

In [456]:
#Remove extra space from the columns now allow to find the correct name
wrench_orders.columns = wrench_orders.columns.str.strip()

In [457]:
#Remove any commas from the Order Qty and convert it to integers
wrench_orders['Order Qty'] = wrench_orders['Order Qty'].replace(',', '', regex=True).astype(int)

C:\Users\valde\AppData\Local\Temp\ipykernel_28256\940646906.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wrench_orders['Order Qty'] = wrench_orders['Order Qty'].replace(',', '', regex=True).astype(int)


In [458]:
#Group these by the month for better visualization later on
wrench_orders_per_month = wrench_orders.groupby(wrench_orders['Order Dt'].dt.to_period('M'))['Order Qty'].sum()

In [459]:
#Display the dataset
display(wrench_orders_per_month)

Order Dt
2014-08    28016
2014-09    31047
2014-10    25415
2014-11    28400
2014-12    42091
2015-01    30045
2015-02    29966
2015-03    41180
2015-04    33189
2015-05    36505
2015-06    45874
2015-07    38937
Freq: M, Name: Order Qty, dtype: int32

In [460]:
#Strip any extra spaces from column names, if necessary
historical_orders.columns = historical_orders.columns.str.strip()

In [461]:
#Convert the 'Order Dt' column to datetime format if it isn't already
historical_orders['Order Dt'] = pd.to_datetime(historical_orders['Order Dt'])

In [462]:
#Filter for hammers (assuming hammers have Product Id = 4)
hammer_orders = historical_orders[historical_orders['Product Id'] == 1]

In [463]:
#Group by store and month, and sum the 'Order Qty' to get the total demand per month per store
hammer_demand_per_month = hammer_orders.groupby([hammer_orders['Str Nbr'], hammer_orders['Order Dt'].dt.to_period('M')])['Order Qty'].sum()


In [464]:
#Print the results
print(hammer_demand_per_month.head(20))

Str Nbr  Order Dt
1        2014-08             4,338  4,598  4,268  3,453 
         2014-09      3,530  3,288  3,999  3,862  3,258 
         2014-10             4,228  3,530  4,227  2,990 
         2014-11             3,493  4,838  3,560  4,329 
         2014-12      4,937  4,361  4,569  4,301  5,106 
         2015-01             4,702  4,031  4,338  4,495 
         2015-02             4,174  4,882  4,099  4,059 
         2015-03      4,212  6,055  5,042  3,806  4,470 
         2015-04             4,799  4,366  4,765  5,134 
         2015-05             4,802  5,394  5,205  5,253 
         2015-06      5,281  5,168  5,038  4,650  5,914 
         2015-07             5,835  5,542  5,355  5,038 
2        2014-08             3,600  3,234  2,623  1,902 
         2014-09      2,567  2,056  3,295  2,569  2,623 
         2014-10             2,897  2,868  3,205  1,470 
         2014-11             2,842  3,750  2,293  3,295 
         2014-12      4,132  3,325  3,508  3,558  4,294 
         2015

In [465]:
#Save for visualization
#hammer_demand_per_month.to_csv('hammer_demand_per_month.csv', index=False)

In [466]:
#Group by store and month, and sum the 'Order Qty' to get the total demand per month for each store
hammer_demand_per_month = hammer_orders.groupby([hammer_orders['Str Nbr'], hammer_orders['Order Dt'].dt.to_period('M')])['Order Qty'].sum()

In [467]:
#Split the data into separate tables for Store 1 and Store 2
store_1_sales = hammer_demand_per_month.loc[1]  # Store 1 sales
store_2_sales = hammer_demand_per_month.loc[2]  # Store 2 sales

In [468]:
#Print the monthly sales for store 1
print("Monthly sales for Store 1:")
print(store_1_sales)

Monthly sales for Store 1:
Order Dt
2014-08            4,338  4,598  4,268  3,453 
2014-09     3,530  3,288  3,999  3,862  3,258 
2014-10            4,228  3,530  4,227  2,990 
2014-11            3,493  4,838  3,560  4,329 
2014-12     4,937  4,361  4,569  4,301  5,106 
2015-01            4,702  4,031  4,338  4,495 
2015-02            4,174  4,882  4,099  4,059 
2015-03     4,212  6,055  5,042  3,806  4,470 
2015-04            4,799  4,366  4,765  5,134 
2015-05            4,802  5,394  5,205  5,253 
2015-06     5,281  5,168  5,038  4,650  5,914 
2015-07            5,835  5,542  5,355  5,038 
Freq: M, Name: Order Qty, dtype: object


In [469]:
#Save for visualization
#store_1_sales.to_csv('store_1_sales.csv', index=False)

In [470]:
#Print the monthly sales for store 2
print("\nMonthly sales for Store 2:")
print(store_2_sales)


Monthly sales for Store 2:
Order Dt
2014-08            3,600  3,234  2,623  1,902 
2014-09     2,567  2,056  3,295  2,569  2,623 
2014-10            2,897  2,868  3,205  1,470 
2014-11            2,842  3,750  2,293  3,295 
2014-12     4,132  3,325  3,508  3,558  4,294 
2015-01            3,319  3,024  2,688  3,448 
2015-02            2,843  4,081  3,390  2,438 
2015-03     2,588  5,147  4,232  2,819  2,809 
2015-04            3,712  3,318  3,689  3,406 
2015-05            3,714  3,944  3,778  4,415 
2015-06     4,453  4,044  4,232  2,982  4,112 
2015-07            4,945  3,776  4,522  3,924 
Freq: M, Name: Order Qty, dtype: object


In [471]:
#Save for visualization
#store_2_sales.to_csv('store_2_sales.csv', index=False)

### Step 2
Total weight per order per store

In [472]:
# Clean up column names by stripping extra spaces
historical_orders.columns = historical_orders.columns.str.strip()

# Filter for wrench
wrench_product_id = product_info[product_info['Product Desc'] == 'Wrench']['Product Id'].iloc[0]

# Filter the historical orders for wrenches
wrench_orders = historical_orders[historical_orders['Product Id'] == wrench_product_id]

# Filter orders by store 1 and store 2
store_1_wrench_orders = wrench_orders[wrench_orders['Str Nbr'] == 1]
store_2_wrench_orders = wrench_orders[wrench_orders['Str Nbr'] == 2]

# Clean column names for wrench orders specifically
wrench_orders.columns = wrench_orders.columns.str.strip()

# Calculate total wrench weight for store 1 and store 2
store_1_total_weight = (store_1_wrench_orders['Order Qty'].str.replace(',', '').astype(float) * store_1_wrench_orders['Product Weight'].astype(float)).sum()
store_2_total_weight = (store_2_wrench_orders['Order Qty'].str.replace(',', '').astype(float) * store_2_wrench_orders['Product Weight'].astype(float)).sum()

# Output the results
print("Total Wrench Weight for Store 1:", store_1_total_weight)
print("Total Wrench Weight for Store 2:", store_2_total_weight)

Total Wrench Weight for Store 1: 516927.4
Total Wrench Weight for Store 2: 386535.6000000001


In [473]:
# Clean up column names by stripping extra spaces
historical_orders.columns = historical_orders.columns.str.strip()

# Filter orders by store 1 and store 2
store_1_orders = historical_orders[historical_orders['Str Nbr'] == 1]
store_2_orders = historical_orders[historical_orders['Str Nbr'] == 2]

# Clean column names for orders specifically
store_1_orders.columns = store_1_orders.columns.str.strip()
store_2_orders.columns = store_2_orders.columns.str.strip()

# Calculate total weight for store 1 and store 2 across all products
store_1_total_weight_all = (store_1_orders['Order Qty'].str.replace(',', '').astype(float) * store_1_orders['Product Weight'].astype(float)).sum()
store_2_total_weight_all = (store_2_orders['Order Qty'].str.replace(',', '').astype(float) * store_2_orders['Product Weight'].astype(float)).sum()

# Output the results
print("Total Product Weight for Store 1 (All Products):", store_1_total_weight_all)
print("Total Product Weight for Store 2 (All Products):", store_2_total_weight_all)


Total Product Weight for Store 1 (All Products): 1775610.9
Total Product Weight for Store 2 (All Products): 1883864.4000000001


In [474]:
# Clean up column names by stripping extra spaces
historical_orders.columns = historical_orders.columns.str.strip()

# Clean the 'Order Qty' column by removing commas and converting to float
historical_orders['Order Qty'] = historical_orders['Order Qty'].str.replace(',', '').astype(float)

# Group by 'Product Id' and 'Str Nbr' (store number), and calculate the total weight
historical_orders['Total Weight'] = historical_orders['Order Qty'] * historical_orders['Product Weight']

# Pivot the data to get a table with 'Product Id' as rows and separate columns for Store 1 and Store 2
product_weight_by_store = historical_orders.pivot_table(index='Product Id', 
                                                        columns='Str Nbr', 
                                                        values='Total Weight', 
                                                        aggfunc='sum')

# Rename columns for clarity
product_weight_by_store.columns = ['Store 1 Total Weight', 'Store 2 Total Weight']

# Reset the index to turn 'Product Id' into a column
product_weight_by_store = product_weight_by_store.reset_index()

# Output the new dataset
print(product_weight_by_store)



   Product Id  Store 1 Total Weight  Store 2 Total Weight
0           1              516927.4              386535.6
1           2              354705.6              370653.6
2           3              903977.9             1126675.2


In [475]:
# Assuming product 1 is the hammer, we will duplicate its row
# Filter the row where 'Product Id' is 1 (hammer)
product_1_data = product_weight_by_store[product_weight_by_store['Product Id'] == 1].copy()

# Modify the 'Product Id' to 4 for the duplicated row (assuming product 4 is the new product)
product_1_data['Product Id'] = 4

# Append the duplicated row to the original dataframe using pd.concat
product_weight_by_store_with_product_4 = pd.concat([product_weight_by_store, product_1_data], ignore_index=True)

# Output the updated dataframe
print(product_weight_by_store_with_product_4)

   Product Id  Store 1 Total Weight  Store 2 Total Weight
0           1              516927.4              386535.6
1           2              354705.6              370653.6
2           3              903977.9             1126675.2
3           4              516927.4              386535.6


In [476]:
# Ensure 'Order Dt' is in datetime format
historical_orders['Order Dt'] = pd.to_datetime(historical_orders['Order Dt'])

# Extract the month and year from 'Order Dt'
historical_orders['Month'] = historical_orders['Order Dt'].dt.to_period('M')

# Convert 'Order Qty' to string first to handle any non-string cases, then remove commas and convert to float
historical_orders['Order Qty'] = historical_orders['Order Qty'].astype(str).str.replace(',', '').astype(float)

# Calculate the total weight for each order
historical_orders['Total Weight'] = historical_orders['Order Qty'] * historical_orders['Product Weight'].astype(float)

# Group by 'Product Id', 'Month', and 'Str Nbr' (store number) and sum the total weights
product_weight_by_month_store = historical_orders.groupby(['Product Id', 'Month', 'Str Nbr'])['Total Weight'].sum().reset_index()

# Pivot the table to get 'Product Id' as rows, 'Month' as columns, and separate columns for Store 1 and Store 2
product_weight_by_month_store_pivot = product_weight_by_month_store.pivot_table(
    index='Product Id', 
    columns=['Month', 'Str Nbr'], 
    values='Total Weight', 
    aggfunc='sum'
)

# Output the result
print(product_weight_by_month_store_pivot)

Month       2014-08           2014-09           2014-10           2014-11  \
Str Nbr           1        2        1        2        1        2        1   
Product Id                                                                  
1           36645.4  24989.8  39461.4  28842.0  32945.0  22968.0  35684.0   
2           26311.2  25239.6  31941.6  34282.8  25587.6  24260.4  27302.4   
3           65677.9  86203.8  83838.3  98604.0  66931.2  80601.3  62797.8   

Month                2014-12            ...  2015-03            2015-04  \
Str Nbr           2        1         2  ...        1         2        1   
Product Id                              ...                               
1           26796.0  51202.8   41397.4  ...  51887.0   38709.0  41940.8   
2           27385.2  33710.4   36658.8  ...  32941.2   35583.6  29592.0   
3           71994.2  82800.8  102023.6  ...  88162.6  114955.0  69388.0   

Month                2015-05           2015-06            2015-07            
Str Nbr 

In [477]:
# Filter historical orders for wrenches (assuming Product Id for wrenches is 1)
wrench_orders = historical_orders[historical_orders['Product Id'] == 1]

# Calculate total wrench orders for each store
store_1_wrench_orders = wrench_orders[wrench_orders['Str Nbr'] == 1]['Order Qty'].sum()
store_2_wrench_orders = wrench_orders[wrench_orders['Str Nbr'] == 2]['Order Qty'].sum()

# Factor in the 10% year-over-year growth
store_1_hammer_orders = store_1_wrench_orders * 1.10
store_2_hammer_orders = store_2_wrench_orders * 1.10

# Total hammer orders for both stores
total_hammer_orders = store_1_hammer_orders + store_2_hammer_orders
print("Total Hammer Orders (Store 1):", store_1_hammer_orders)
print("Total Hammer Orders (Store 2):", store_2_hammer_orders)
print("Total Hammer Orders (Both Stores):", total_hammer_orders)

Total Hammer Orders (Store 1): 258463.7
Total Hammer Orders (Store 2): 193267.80000000002
Total Hammer Orders (Both Stores): 451731.5


In [478]:
# Ensure 'Order Dt' and 'Shipment Date' are in datetime format
historical_orders['Order Dt'] = pd.to_datetime(historical_orders['Order Dt'])
historical_shipments['Shipment Date'] = pd.to_datetime(historical_shipments['Shipment Date'])

# Strip extra spaces from column names in both dataframes
historical_orders.columns = historical_orders.columns.str.strip()
historical_shipments.columns = historical_shipments.columns.str.strip()

# Merge the orders and shipments data on store number and supplier to match the shipment to the order
# We'll use 'Str Nbr' (store number) and 'Supplier' for merging.
orders_shipments_data = pd.merge(historical_orders, historical_shipments, on=['Str Nbr', 'Supplier'], how='left', suffixes=('_order', '_shipment'))

# Select the relevant columns: Shipment Date, Order Date, Store Number, Supplier, Order Quantity, Product Weight, Shipment Weight
past_orders_shipments = orders_shipments_data[['Shipment Date', 'Order Dt', 'Str Nbr', 'Supplier', 'Order Qty', 'Product Weight', 'Weight']]

# Sort the data by Shipment Date for easier viewing
past_orders_shipments_sorted = past_orders_shipments.sort_values(by='Shipment Date')

# Output the past orders and shipments
print(past_orders_shipments_sorted)


      Shipment Date   Order Dt  Str Nbr Supplier  Order Qty  Product Weight  \
0        2014-08-04 2014-08-04        1        A     4338.0             2.2   
8008     2014-08-04 2015-01-26        1        B     1742.0             8.3   
8060     2014-08-04 2015-01-26        2        B     2311.0             8.3   
8112     2014-08-04 2015-02-02        1        A     4174.0             2.2   
8164     2014-08-04 2015-02-02        1        A     5324.0             1.2   
...             ...        ...      ...      ...        ...             ...   
10711    2015-07-27 2015-03-30        1        A     5517.0             1.2   
10763    2015-07-27 2015-03-30        2        A     2809.0             2.2   
10815    2015-07-27 2015-03-30        2        A     6551.0             1.2   
10451    2015-07-27 2015-03-23        2        A     2819.0             2.2   
16223    2015-07-27 2015-07-27        2        B     2888.0             8.3   

       Weight  
0       15551  
8008    15687  
806

In [479]:
# Load the historical shipments dataset, remove 'Unnamed: 4' column if present
historical_shipments = pd.read_csv('Historical_shipments.csv').drop(columns=['Unnamed: 4'], errors='ignore')

# Clean up the column names by stripping any leading or trailing spaces
historical_shipments.columns = historical_shipments.columns.str.strip()

# Convert the 'Weight' column to numeric (float) to handle any non-numeric issues (e.g., commas, spaces)
historical_shipments['Weight'] = pd.to_numeric(historical_shipments['Weight'].replace({',': ''}, regex=True), errors='coerce')

# Define the carrier costs based on the store and supplier
carrier_costs = {
    ('X', 'A', 1): 2540,  # Carrier X, Supplier A, Store 1 (Flat rate)
    ('X', 'A', 2): 1640,  # Carrier X, Supplier A, Store 2 (Flat rate)
    ('X', 'B', 1): 1200,  # Carrier X, Supplier B, Store 1 (Flat rate)
    ('X', 'B', 2): 1200,  # Carrier X, Supplier B, Store 2 (Flat rate)
    ('Y', 'A', 1): 0.12,  # Carrier Y, Supplier A, Store 1 (Variable rate per pound)
    ('Y', 'A', 2): 0.07,  # Carrier Y, Supplier A, Store 2 (Variable rate per pound)
    ('Y', 'B', 1): 0.07,  # Carrier Y, Supplier B, Store 1 (Variable rate per pound)
    ('Y', 'B', 2): 0.06,  # Carrier Y, Supplier B, Store 2 (Variable rate per pound)
}

# Calculate Carrier X Cost (Flat Rate) for each row
historical_shipments['Carrier X Cost'] = historical_shipments.apply(
    lambda row: carrier_costs[('X', row['Supplier'], row['Str Nbr'])], axis=1)

# Calculate Carrier Y Cost (Variable Rate based on weight) for each row
historical_shipments['Carrier Y Cost'] = historical_shipments.apply(
    lambda row: row['Weight'] * carrier_costs[('Y', row['Supplier'], row['Str Nbr'])], axis=1)

# Display the first few rows of the dataset to verify
print(historical_shipments.head())

  Shipment Date  Str Nbr Supplier  Weight  Carrier X Cost  Carrier Y Cost
0    2014-08-04        1        A   15551            2540         1866.12
1    2014-08-04        1        B   15687            1200         1098.09
2    2014-08-04        2        A   14128            1640          988.96
3    2014-08-04        2        B   21049            1200         1262.94
4    2014-08-11        1        A   16671            2540         2000.52


In [480]:
# Clean up the column names by stripping any leading or trailing spaces
historical_orders.columns = historical_orders.columns.str.strip()
historical_shipments.columns = historical_shipments.columns.str.strip()

# Convert the 'Weight' column in shipments to numeric (handle commas, spaces)
historical_shipments['Weight'] = pd.to_numeric(historical_shipments['Weight'].replace({',': ''}, regex=True), errors='coerce')

# Filter for Product Id 1 (wrenches, which will be treated as hammers weighing 2 pounds per unit)
wrench_orders = historical_orders[historical_orders['Product Id'] == 1]

# Convert the 'Order Qty' to numeric and calculate hammer weight as 2 pounds per item
wrench_orders['Order Qty'] = pd.to_numeric(wrench_orders['Order Qty'].replace({',': ''}, regex=True), errors='coerce')
wrench_orders['Hammer Weight'] = wrench_orders['Order Qty'] * 2

print(wrench_orders.head())

     Order Dt  Str Nbr Supplier  Product Id  Order Qty  Product Weight  \
0  2014-08-04        1        A           1     4338.0             2.2   
2  2014-08-04        2        A           1     3600.0             2.2   
6  2014-08-11        1        A           1     4598.0             2.2   
8  2014-08-11        2        A           1     3234.0             2.2   
12 2014-08-18        1        A           1     4268.0             2.2   

   Order Weight  Total Weight    Month  Hammer Weight  
0        9,544         9543.6  2014-08         8676.0  
2        7,920         7920.0  2014-08         7200.0  
6       10,116        10115.6  2014-08         9196.0  
8        7,115         7114.8  2014-08         6468.0  
12       9,390         9389.6  2014-08         8536.0  


C:\Users\valde\AppData\Local\Temp\ipykernel_28256\2856382155.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wrench_orders['Order Qty'] = pd.to_numeric(wrench_orders['Order Qty'].replace({',': ''}, regex=True), errors='coerce')
C:\Users\valde\AppData\Local\Temp\ipykernel_28256\2856382155.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wrench_orders['Hammer Weight'] = wrench_orders['Order Qty'] * 2


In [481]:
# Ensure the 'Order Dt' column is in datetime format
historical_orders['Order Dt'] = pd.to_datetime(historical_orders['Order Dt'])

# Filter for hammers (assuming Product Id = 4 for hammers)
hammer_orders = historical_orders[historical_orders['Product Id'] == 1]

print(hammer_orders)

# Group by store and sum the 'Order Qty' for each store to get the total orders for the year
total_orders_store_1 = hammer_orders[hammer_orders['Str Nbr'] == 1]['Order Qty'].sum()
total_orders_store_2 = hammer_orders[hammer_orders['Str Nbr'] == 2]['Order Qty'].sum()

# Calculate the total number of hammers ordered for both stores combined
total_hammers_ordered = total_orders_store_1 + total_orders_store_2

# Print the total number of hammers ordered for the entire year
print(f"Total hammers ordered for the entire year: {total_hammers_ordered}")


      Order Dt  Str Nbr Supplier  Product Id  Order Qty  Product Weight  \
0   2014-08-04        1        A           1     4338.0             2.2   
2   2014-08-04        2        A           1     3600.0             2.2   
6   2014-08-11        1        A           1     4598.0             2.2   
8   2014-08-11        2        A           1     3234.0             2.2   
12  2014-08-18        1        A           1     4268.0             2.2   
..         ...      ...      ...         ...        ...             ...   
296 2015-07-13        2        A           1     3776.0             2.2   
300 2015-07-20        1        A           1     5355.0             2.2   
302 2015-07-20        2        A           1     4522.0             2.2   
306 2015-07-27        1        A           1     5038.0             2.2   
308 2015-07-27        2        A           1     3924.0             2.2   

    Order Weight  Total Weight    Month  
0         9,544         9543.6  2014-08  
2         7,920

In [482]:
# Step 1: Find the Product Id for Wrenches in the 'Product Info' dataset
wrench_product_id = product_info[product_info['Product Desc'].str.lower() == 'wrench']['Product Id'].values[0]

# Step 2: Filter the Historical Orders dataset for only wrench orders based on the Product Id
wrench_orders = historical_orders[historical_orders['Product Id'] == wrench_product_id]

# Step 3: Check if 'Order Qty' is string type, remove commas if necessary, then convert to integer
if wrench_orders['Order Qty'].dtype == 'object':
    wrench_orders['Order Qty'] = wrench_orders['Order Qty'].str.replace(',', '').astype(int)
else:
    wrench_orders['Order Qty'] = wrench_orders['Order Qty'].astype(int)

# Step 4: Calculate the total number of wrenches sold
total_wrenches_sold = wrench_orders['Order Qty'].sum()

total_wrenches_sold


C:\Users\valde\AppData\Local\Temp\ipykernel_28256\2398328973.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wrench_orders['Order Qty'] = wrench_orders['Order Qty'].astype(int)


410665

In [483]:
# Step 3: Strip leading and trailing spaces from column names
historical_orders.columns = historical_orders.columns.str.strip()

# Step 4: Group by 'Product Id' and sum 'Order Qty' for all months combined
total_product_purchases = historical_orders.groupby('Product Id')['Order Qty'].sum().reset_index()

# Step 5: Rename the column for clarity
total_product_purchases.columns = ['Product Id', 'Total Order Qty']

# Step 6: Output or save the results
print(total_product_purchases)

   Product Id  Total Order Qty
0           1         410665.0
1           2         604466.0
2           3         244657.0


In [487]:
# Step 3: Strip leading and trailing spaces from column names
historical_orders.columns = historical_orders.columns.str.strip()

# Step 4: Convert 'Order Dt' to datetime format
historical_orders['Order Dt'] = pd.to_datetime(historical_orders['Order Dt'])

# Step 5: Create a new column for 'Year-Month'
historical_orders['Year-Month'] = historical_orders['Order Dt'].dt.to_period('M')

# Step 6: Group by 'Year-Month' and 'Product Id', summing 'Order Qty'
# Ensure 'Order Qty' is numeric, converting any non-numeric values to NaN
historical_orders['Order Qty'] = pd.to_numeric(historical_orders['Order Qty'], errors='coerce')

monthly_product_purchases = historical_orders.groupby(['Year-Month', 'Product Id'])['Order Qty'].sum().reset_index()

# Step 7: Apply a 10% increase to the 'Order Qty' column, handling NaN values
monthly_product_purchases['Order Qty'] = monthly_product_purchases['Order Qty'].fillna(0)

# Step 8: Output or save the results
print(monthly_product_purchases)

   Year-Month  Product Id  Order Qty
0     2014-08           1    28016.0
1     2014-08           2    42959.0
2     2014-08           3    18299.0
3     2014-09           1    31047.0
4     2014-09           2    55187.0
5     2014-09           3    21981.0
6     2014-10           1    25415.0
7     2014-10           2    41540.0
8     2014-10           3    17775.0
9     2014-11           1    28400.0
10    2014-11           2    45573.0
11    2014-11           3    16240.0
12    2014-12           1    42091.0
13    2014-12           2    58641.0
14    2014-12           3    22268.0
15    2015-01           1    30045.0
16    2015-01           2    39027.0
17    2015-01           3    16264.0
18    2015-02           1    29966.0
19    2015-02           2    42512.0
20    2015-02           3    19007.0
21    2015-03           1    41180.0
22    2015-03           2    57104.0
23    2015-03           3    24472.0
24    2015-04           1    33189.0
25    2015-04           2    52070.0
2